In [1]:
import nibabel as nib

file_path = '/home/sastocke/nnUNet/nnUNet_raw/Dataset001_LeDirectionsvsAveragesDataOnlyGood/imagesTr/Le_Volunteer_01_DiVO_06_10_slice_001_0000.nii.gz'
img = nib.load(file_path)
print(img.shape)  # This should show the shape of your NIfTI file


(256, 256)


In [2]:
import SimpleITK as sitk
import nibabel as nib

# Using SimpleITK
try:
    image_sitk = sitk.ReadImage('/home/sastocke/nnUNet/nnUNet_raw/Dataset001_LeDirectionsvsAveragesDataOnlyGood/imagesTr/Le_Volunteer_01_DiVO_06_10_slice_001_0000.nii.gz')
    print('SimpleITK: Image size:', image_sitk.GetSize())
except Exception as e:
    print('SimpleITK error:', e)

# Using Nibabel
try:
    image_nib = nib.load('/home/sastocke/nnUNet/nnUNet_raw/Dataset001_LeDirectionsvsAveragesDataOnlyGood/imagesTr/Le_Volunteer_01_DiVO_06_10_slice_001_0000.nii.gz')
    print('Nibabel: Image shape:', image_nib.shape)
except Exception as e:
    print('Nibabel error:', e)


SimpleITK: Image size: (256, 256)
Nibabel: Image shape: (256, 256)


In [1]:
import os

# Set the paths to your imagesTr and labelsTr directories
image_dir = '/Users/saschastocker/Documents/Stanford/work2024/WholeHeartCropISMRM/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/imagesTr/'
label_dir = '/Users/saschastocker/Documents/Stanford/work2024/WholeHeartCropISMRM/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/labelsTr/'

# Rename images: Preserve slice info and add '_0000'
def rename_images(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii'):
            new_filename = filename.replace('_image.nii', '_0000.nii.gz')  # Add '_0000' before .nii
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f'Renamed: {filename} -> {new_filename}')

# Rename labels: Preserve slice info and remove _image if present
def rename_labels(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii'):
            new_filename = filename.replace('_mask.nii', '.nii.gz')  # Remove '_image' if present
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f'Renamed: {filename} -> {new_filename}')

# Apply renaming to images and labels
rename_images(image_dir)
rename_labels(label_dir)


Renamed: Le_Volunteer_10_DiVO_10_06_slice_002_image.nii -> Le_Volunteer_10_DiVO_10_06_slice_002_0000.nii.gz
Renamed: Le_Volunteer_10_DiVO_10_06_slice_001_image.nii -> Le_Volunteer_10_DiVO_10_06_slice_001_0000.nii.gz
Renamed: Le_Volunteer_11_DiVO_12_05_slice_001_image.nii -> Le_Volunteer_11_DiVO_12_05_slice_001_0000.nii.gz
Renamed: Le_Volunteer_10_DiVO_15_04_slice_001_image.nii -> Le_Volunteer_10_DiVO_15_04_slice_001_0000.nii.gz
Renamed: Le_Volunteer_10_DiVO_06_10_slice_001_image.nii -> Le_Volunteer_10_DiVO_06_10_slice_001_0000.nii.gz
Renamed: Le_Volunteer_01_DiVO_06_10_slice_003_image.nii -> Le_Volunteer_01_DiVO_06_10_slice_003_0000.nii.gz
Renamed: Le_Volunteer_11_DiVO_15_04_slice_002_image.nii -> Le_Volunteer_11_DiVO_15_04_slice_002_0000.nii.gz
Renamed: Le_Volunteer_06_DiVO_06_10_slice_003_image.nii -> Le_Volunteer_06_DiVO_06_10_slice_003_0000.nii.gz
Renamed: Le_Volunteer_04_DiVO_12_05_slice_003_image.nii -> Le_Volunteer_04_DiVO_12_05_slice_003_0000.nii.gz
Renamed: Le_Volunteer_12_DiV

In [4]:
import os

# Set the paths to your imagesTr and labelsTr directories
image_dir = '/home/sastocke/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/imagesTr/'
label_dir = '/home/sastocke/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/labelsTr/'

# Rename images: Preserve slice info and add '_0000'
def rename_images(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii.gz'):
            new_filename = filename.replace('.nii.gz', '.nii.')  # Add '_0000' before .nii
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f'Renamed: {filename} -> {new_filename}')

# Rename labels: Preserve slice info and remove _image if present
def rename_labels(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii.gz'):
            new_filename = filename.replace('.nii.gz', '.nii.')  # Remove '_image' if present
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f'Renamed: {filename} -> {new_filename}')

# Apply renaming to images and labels
rename_images(image_dir)
rename_labels(label_dir)

Renamed: Le_Volunteer_11_DiVO_06_10_slice_003_0000.nii.gz -> Le_Volunteer_11_DiVO_06_10_slice_003_0000.nii.
Renamed: Le_Volunteer_07_DiVO_06_10_slice_001_0000.nii.gz -> Le_Volunteer_07_DiVO_06_10_slice_001_0000.nii.
Renamed: Le_Volunteer_04_DiVO_12_05_slice_001_0000.nii.gz -> Le_Volunteer_04_DiVO_12_05_slice_001_0000.nii.
Renamed: Le_Volunteer_10_DiVO_15_04_slice_001_0000.nii.gz -> Le_Volunteer_10_DiVO_15_04_slice_001_0000.nii.
Renamed: Le_Volunteer_05_DiVO_30_02_slice_002_0000.nii.gz -> Le_Volunteer_05_DiVO_30_02_slice_002_0000.nii.
Renamed: Le_Volunteer_02_DiVO_12_05_slice_001_0000.nii.gz -> Le_Volunteer_02_DiVO_12_05_slice_001_0000.nii.
Renamed: Le_Volunteer_03_DiVO_15_04_slice_002_0000.nii.gz -> Le_Volunteer_03_DiVO_15_04_slice_002_0000.nii.
Renamed: Le_Volunteer_02_DiVO_20_03_slice_001_0000.nii.gz -> Le_Volunteer_02_DiVO_20_03_slice_001_0000.nii.
Renamed: Le_Volunteer_08_DiVO_15_04_slice_002_0000.nii.gz -> Le_Volunteer_08_DiVO_15_04_slice_002_0000.nii.
Renamed: Le_Volunteer_02_MDD

In [6]:
import os
import nibabel as nib

# Function to check the shape of the NIfTI files
def check_nifti_shapes(directory):
    mismatched_files = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".nii") or file.endswith(".nii.gz"):
                file_path = os.path.join(root, file)
                img = nib.load(file_path)
                shape = img.shape
                
                # Check if the shape matches expected (5, 256, 256)
                if shape != (5, 256, 256):
                    mismatched_files.append((file, shape))
    
    return mismatched_files

# Example usage
directory = '/home/sastocke/nnUNet/nnUNet_raw/Dataset003_LeCombinedEigenvalue1making3channels/labelsTr/' # Replace with your actual path
mismatched_files = check_nifti_shapes(directory)

if mismatched_files:
    for file, shape in mismatched_files:
        print(f"File {file} has shape {shape}, expected (5, 256, 256)")
else:
    print("All files have the correct shape.")


File Le_Volunteer_08_DiVO_06_10_slice_003.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_11_DiVO_15_04_slice_003.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_01_MDDW_12_05_slice_002.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_01_DiVO_20_03_slice_001.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_02_DiVO_30_02_slice_001.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_09_DiVO_30_02_slice_003.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_06_DiVO_12_05_slice_002.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_06_DiVO_10_06_slice_001.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_12_DiVO_30_02_slice_002.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_08_DiVO_12_05_slice_003.nii.gz has shape (256, 256), expected (5, 256, 256)
File Le_Volunteer_09_DiVO_15_04_slice_003.nii.gz has shape (

In [1]:
import os
import nibabel as nib


# Set the paths to your imagesTr and labelsTr directories
image_dir = '/home/sastocke/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/imagesTr/'
label_dir = '/home/sastocke/nnUNet/nnUNet_raw/Dataset002_CombinedLeDirectionsvsAverages/labelsTr/'

import os
import nibabel as nib

# Function to rename and save images with nibabel
def rename_and_compress_images(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii.'):
            old_filepath = os.path.join(directory, filename)
            new_filename = filename.replace('nii.', '.nii')  # Add '_0000' before .nii.gz
            new_filepath = os.path.join(directory, new_filename)
            
            # Load the NIfTI file using nibabel
            img = nib.load(old_filepath)
            
            # Save it as compressed .nii.gz using nibabel
            nib.save(img, new_filepath)
            
            # Remove the old .nii file if it was successfully compressed
            os.remove(old_filepath)
            print(f'Renamed and compressed: {filename} -> {new_filename}')

# Function to rename and compress labels with nibabel
def rename_and_compress_labels(directory):
    for filename in os.listdir(directory):
        if 'slice' in filename and filename.endswith('.nii.gz'):
            old_filepath = os.path.join(directory, filename)
            new_filename = filename.replace('_.nii.gz', '.nii.gz')  # Remove '_mask' and compress
            new_filepath = os.path.join(directory, new_filename)
            
            # Load the NIfTI file using nibabel
            img = nib.load(old_filepath)
            
            # Save it as compressed .nii.gz using nibabel
            nib.save(img, new_filepath)
            
            # Remove the old .nii file if it was successfully compressed
            os.remove(old_filepath)
            print(f'Renamed and compressed: {filename} -> {new_filename}')
# Apply renaming and compressing to images and labels
#rename_and_compress_images(image_dir)
rename_and_compress_labels(label_dir)


Renamed and compressed: Le_Volunteer_03_DiVO_30_02_slice_001_.nii.gz -> Le_Volunteer_03_DiVO_30_02_slice_001.nii.gz
Renamed and compressed: Le_Volunteer_02_DiVO_30_02_slice_002_.nii.gz -> Le_Volunteer_02_DiVO_30_02_slice_002.nii.gz
Renamed and compressed: Le_Volunteer_05_DiVO_30_02_slice_003_.nii.gz -> Le_Volunteer_05_DiVO_30_02_slice_003.nii.gz
Renamed and compressed: Le_Volunteer_06_DiVO_15_04_slice_001_.nii.gz -> Le_Volunteer_06_DiVO_15_04_slice_001.nii.gz
Renamed and compressed: Le_Volunteer_10_DiVO_20_03_slice_001_.nii.gz -> Le_Volunteer_10_DiVO_20_03_slice_001.nii.gz
Renamed and compressed: Le_Volunteer_08_DiVO_10_06_slice_003_.nii.gz -> Le_Volunteer_08_DiVO_10_06_slice_003.nii.gz
Renamed and compressed: Le_Volunteer_06_DiVO_12_05_slice_001_.nii.gz -> Le_Volunteer_06_DiVO_12_05_slice_001.nii.gz
Renamed and compressed: Le_Volunteer_02_MDDW_06_10_slice_003_.nii.gz -> Le_Volunteer_02_MDDW_06_10_slice_003.nii.gz
Renamed and compressed: Le_Volunteer_06_DiVO_06_10_slice_001_.nii.gz -> 

In [7]:
import os
import nibabel as nib

# Set the paths to your imagesTr and labelsTr directories
image_dir  = '/home/sastocke/nnUNet/nnUNet_raw/Dataset003_LeCombinedEigenvalue1making3channels/labelsTr/'
# Function to fix dimension order
def fix_dimension_order(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.nii.gz'):
            filepath = os.path.join(directory, filename)
            
            # Load the NIfTI file
            img = nib.load(filepath)
            data = img.get_fdata()  # Get the image data
            
            # Check if the shape is in the wrong order
            if data.shape == (256, 256, 5):
                # Transpose the dimensions to (5, 256, 256)
                fixed_data = data.transpose(2, 0, 1)
                
                # Create a new NIfTI image with the corrected dimensions
                fixed_img = nib.Nifti1Image(fixed_data, img.affine, img.header)
                
                # Save the corrected image (overwrite the original file)
                nib.save(fixed_img, filepath)
                print(f'Fixed and saved: {filename}')
            else:
                print(f'File {filename} already has correct shape: {data.shape}')

# Apply the fix to the image directory
fix_dimension_order(image_dir)


File Le_Volunteer_08_DiVO_06_10_slice_003.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_11_DiVO_15_04_slice_003.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_01_MDDW_12_05_slice_002.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_01_DiVO_20_03_slice_001.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_02_DiVO_30_02_slice_001.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_09_DiVO_30_02_slice_003.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_06_DiVO_12_05_slice_002.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_06_DiVO_10_06_slice_001.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_12_DiVO_30_02_slice_002.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_08_DiVO_12_05_slice_003.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_09_DiVO_15_04_slice_003.nii.gz already has correct shape: (256, 256)
File Le_Volunteer_08_DiVO_10_06_slice_001.n

In [8]:
import os
import nibabel as nib
import numpy as np

# Set the path to your labels directory
label_dir  = '/home/sastocke/nnUNet/nnUNet_raw/Dataset003_LeCombinedEigenvalue1making3channels/labelsTr/'

# Function to add a channel dimension to labels
def extend_labels(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.nii.gz'):
            filepath = os.path.join(directory, filename)
            img = nib.load(filepath)
            data = img.get_fdata()

            # If the label doesn't already have a channel dimension, add one
            if data.shape == (256, 256):  # Only extend labels if necessary
                extended_data = np.expand_dims(data, axis=0)  # Add channel dimension
                new_img = nib.Nifti1Image(extended_data, img.affine, img.header)
                
                # Save the new label file, overwriting the original
                nib.save(new_img, filepath)
                print(f'Extended {filename} to shape {extended_data.shape}')

# Apply to the labels directory
extend_labels(label_dir)


Extended Le_Volunteer_08_DiVO_06_10_slice_003.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_11_DiVO_15_04_slice_003.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_01_MDDW_12_05_slice_002.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_01_DiVO_20_03_slice_001.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_02_DiVO_30_02_slice_001.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_09_DiVO_30_02_slice_003.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_06_DiVO_12_05_slice_002.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_06_DiVO_10_06_slice_001.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_12_DiVO_30_02_slice_002.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_08_DiVO_12_05_slice_003.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_09_DiVO_15_04_slice_003.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_08_DiVO_10_06_slice_001.nii.gz to shape (1, 256, 256)
Extended Le_Volunteer_06_DiVO_06_10_slice_002.nii.gz to shape (1, 256, 256)
Extended Le_

In [3]:
import os
import nibabel as nib 
import numpy as np# Input directories
lv_dir = "/home/sastocke/nnUNet/HannumTestDataPrepared/TestDatasetLVOnly/labelsTr"
ip_dir = "/home/sastocke/nnUNet/HannumTestDataPrepared/TestDatasetIPOnly/labelsTr"
output_dir = "/home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process files
for file in os.listdir(lv_dir):
    if file.endswith(".nii.gz"):
        lv_path = os.path.join(lv_dir, file)
        ip_path = os.path.join(ip_dir, file)
        output_path = os.path.join(output_dir, file)
        
        # Load the LV and IP masks
        lv_img = nib.load(lv_path)
        ip_img = nib.load(ip_path)
        
        lv_data = lv_img.get_fdata().astype(np.int32)
        ip_data = ip_img.get_fdata().astype(np.int32)
        
        # Combine masks according to rules
        combined_data = lv_data.copy()
        combined_data[lv_data == 1] = 1  # LV remains 1
        combined_data[ip_data == 1] = 2  # IP label 1 -> 2
        combined_data[ip_data == 2] = 3  # IP label 2 -> 3
        
        # Save the combined mask
        combined_img = nib.Nifti1Image(combined_data, affine=lv_img.affine, header=lv_img.header)
        nib.save(combined_img, output_path)
        print(f"Processed and saved: {output_path}")

print("All masks have been combined and saved to Output_Final_Combined folder.")

Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_11_MDDW_06_10_slice_002.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_11_MDDW_06_10_slice_003.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_09_MDDW_06_10_slice_002.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_12_MDDW_06_10_slice_001.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_10_MDDW_06_10_slice_003.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_09_MDDW_06_10_slice_001.nii.gz
Processed and saved: /home/sastocke/nnUNet/HannumTestDataPrepared/GTLVandIPsSeperate/HannumTestData_Volunteer_10_MDDW_06_10_slice_001.nii.gz
Processed and